# Cleaning, Reshaping and Plotting Data with Python

This is a [Jupyter](http://jupyter.org/) notebook that shows how to use Python and Pandas to clean up a slightly dirty dataset for analysis. We'll also look at using [matplotlib](https://matplotlib.org/) for exploratory charting to help us understand and find stories in the data.

We hope to cover:

* Cleaning data
    * Renaming columns
    * Removing rows and columns
    * Merging header columns that span multiple rows
    * Converting the type of variables in a dataset
    * Finding and fixing bad values
* Exploratory visualization
    * Reshaping data for visualization
    * [Using Pandas with matplotlib](https://pandas.pydata.org/pandas-docs/stable/visualization.html) to make simple plots
    
## Keep this in mind

One of the hardest things that I found when learning Pandas was that there are many ways of doing similar things. The *best* way often depends on exactly what you are trying to do and how important performance is. I'll show multiple methods of doing things where it feels absolutely neccessary, but for brevity and clarity, I'm going to try to stick to one method. There are other ways of doing these things that might be better for a particular situaton.

## Python stuff we'll use

You will get the most out of this session if you have some familiarity with Python programming and the basics of working with Pandas. There are a few Python concepts that you'll see in the code. You don't have to have extensive experience using them, but it will help if you know that they exist and how to spot them in the code.

* Defining functions in Python.
* "keyword" and "positional" arguments of functions.
* [List comprehensions](http://www.pythonforbeginners.com/basics/list-comprehensions-in-python) and [dict comprehensions](https://www.python.org/dev/peps/pep-0274/)
* [Regular expressions](https://docs.python.org/3/howto/regex.html)
* [`str.format()`](https://pyformat.info/)
* `try`/`except` and [exception handling](https://docs.python.org/3/tutorial/errors.html)

## Pandas stuff we'll use

* The `DataFrame` and `Series` data structures and understanding the relationship between the two.
* Understanding what the concept of an Index of a `DataFrame` or `Series`.
* Understanding that many `DataFrame` methods return a new `DataFrame`
* Accessing `DataFrame` values with [`at`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.at.html#pandas.DataFrame.at), [`loc`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html#pandas.DataFrame.loc), [`iat`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.iat.html#pandas.DataFrame.iat) and [`iloc`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.iloc.html#pandas.DataFrame.iloc). The best starting place for this is probably the [Indexing and Selecting Data](https://pandas.pydata.org/pandas-docs/stable/indexing.html) section of the Pandas docs.
* Showing only a few rows of a `DataFrame` using [`DataFrame.head`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html)


## Notebook boilerplate

There are a few things that you'll do when you initialize a Jupyter notebook to work with Pandas. This includes importing pandas and specifying how we'll display plots in our notebook. Many of your data cleaning and visualization notebooks or programs will will start out this way.

In [261]:
# Specify how `matplotlib` will display plots in this notebook.
# `notebook` will create interactive charts.
# `inline` will create static images.
%matplotlib notebook
#%matplotlib inline

# It's conventional to import pandas as `pd`
import pandas as pd

# This is the conventional way to import the matplotlib API
import matplotlib.pyplot as plt

# We'll explicitly display dataframes inline in our notebook
from IPython.display import display

# The `os.path` module that comes as part of Python's standard library helps us build pathnames.
# We'll use this to build variables that point to our data.
import os

# Data cleaning will almost surely use regular expressions at some point
import re

This is some code we'll use throughout the notebook to make it easier to use as a learning tool. You probably won't need to do this in your own notebooks.

In [262]:
from IPython.utils.coloransi import TermColors as tc

def print_red(s):
    print(tc.Red + s + tc.Normal)
    
def print_green(s):
    print(tc.Green + s + tc.Normal)

## Data cleaning

### Take a peek at the data

Let's load some data to identify some potential issues with the data.

In [263]:
# Create some variables to point to our data
BASE_PATH = os.getcwd()
DATA_PATH = os.path.join(BASE_PATH, 'data')

# Define a helper function to get the path to a particular month's CSV file.
def get_monthly_data_path(month):
    """Returns a path to a month's CSV file"""
    return os.path.join(DATA_PATH, 'CBC_{}_2017_Disposition_Stats.csv'.format(month))


In [264]:
data_path = get_monthly_data_path('January')

# I like to store the data that is loaded without any additional transformations in a `_raw` variable.
# This lets me try different transforms and throw them away if they don't work without having to
# reload the data.
data_january_raw = pd.read_csv(data_path)

display(data_january_raw)

,Jan-17,103,606,604,605,277,601,153,TOTAL CASES,894,278,279,RELS'D ON,RELS'D ON D
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C BOND,BOND
1,011/2017,0.0,4.0,0.0,45.0,12.0,64,0.0,125,0.0,0.0,0,1,67
2,1/2/2017,0.0,4.0,0.0,37.0,9.0,68,3.0,121,0.0,0.0,0,3,63
3,1/3/2017,0.0,3.0,0.0,21.0,10.0,39,0.0,73,0.0,0.0,0,0,23
4,1/4/2017,0.0,6.0,0.0,27.0,20.0,41,0.0,94,0.0,1.0,2,0,32
5,1/5/2017,0.0,8.0,0.0,24.0,7.0,53,0.0,92,0.0,0.0,0,0,46
6,1/6/2017,0.0,10.0,0.0,20.0,9.0,43,0.0,82,0.0,0.0,0,0,33
7,1/7/2017,0.0,7.0,4.0,39.0,18.0,53,0.0,121,0.0,0.0,0,0,56
8,1/8/2017,0.0,2.0,0.0,24.0,26.0,78,1.0,131,0.0,0.0,0,1,72
9,1/9/2017,2.0,6.0,0.0,26.0,11.0,38,0.0,83,0.0,22.0,20,1,73


### What's wrong with the data?


What things might be problematic about this data? Think about it and then scroll down to see the answers.

⬇️

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

* Column name of first column reflects the month of the data. Why is this a problem?
* Codes in column names
* Headers in columns 13 and 14 split across rows
* Malformed date for first row of data
* `TOTALS` row that doesn't exactly represent the same time unit as the other rows
* A value of `q0` in the `278` column

To better understand how this dirtyness occurred, it might be helpful to understand what happened before we loaded this data into Pandas. This data originally came as a PDF file, that was converted to CSV using [Tabula](http://tabula.technology/).

Here's what the January data looks like:

![January data as a PDF](img/screenshot__january_data.png)

As you can see, the data issues are due to a combination of factors:

* data designed for humans to read, not computers
* human error when entering data
* issues introduced when converting the data from PDF

Regardless of the cause, we'll try to clean up the data and address the issues we identified earlier one by one as we move forward.

### Renaming columns

Let's rename the first column in a way that might let us more easily combine the monthly data into one dataset.

We can use the [`pandas.DataFrame.rename`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html) method to do this.

We're specifying our renaming mapping by passing a `dict` where the keys are the old column names and the values are the new column names.

We specify that we're renaming columns of the spreadsheet rather than rows by specifying the `columns` keyword argument. `DataFrame.rename` can also be used to rename index entries.

In [265]:
data_january = data_january_raw.rename(columns={'Jan-17': 'date'})
display(data_january.head())

,date,103,606,604,605,277,601,153,TOTAL CASES,894,278,279,RELS'D ON,RELS'D ON D
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C BOND,BOND
1,011/2017,0.0,4.0,0.0,45.0,12.0,64,0.0,125,0.0,0.0,0,1,67
2,1/2/2017,0.0,4.0,0.0,37.0,9.0,68,3.0,121,0.0,0.0,0,3,63
3,1/3/2017,0.0,3.0,0.0,21.0,10.0,39,0.0,73,0.0,0.0,0,0,23
4,1/4/2017,0.0,6.0,0.0,27.0,20.0,41,0.0,94,0.0,1.0,2,0,32


We can also rename the columns by assigning a new list of columns to the `DataFrame.columns` attribute. This isn't the ideal way to do it, because it's less clear what your code is doing than with using `rename`, but there are some cases where it's useful, and easier.

In [266]:
# Get the existing column names
col_names = data_january_raw.columns.values.tolist()
# This also works, but the above method is the more performant method:
# col_names = list(data_january_raw)

# Change the first column name
col_names[0] = 'date'

# Make a copy of the original dataframe
data_january = data_january_raw.copy()

# Set the column names of the new dataframe to our modified names
data_january.columns = col_names

display(data_january.head())

,date,103,606,604,605,277,601,153,TOTAL CASES,894,278,279,RELS'D ON,RELS'D ON D
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C BOND,BOND
1,011/2017,0.0,4.0,0.0,45.0,12.0,64,0.0,125,0.0,0.0,0,1,67
2,1/2/2017,0.0,4.0,0.0,37.0,9.0,68,3.0,121,0.0,0.0,0,3,63
3,1/3/2017,0.0,3.0,0.0,21.0,10.0,39,0.0,73,0.0,0.0,0,0,23
4,1/4/2017,0.0,6.0,0.0,27.0,20.0,41,0.0,94,0.0,1.0,2,0,32


### Renaming columns: you try it

The columns in the source data use codes, which don't mean a whole lot to us. Luckily, the souce PDFs define a lookup table to titles that are easier to understand. We can easily make a Python dictionary out of it.

In [267]:
CODE_LOOKUP = {
    '103': "Nolle Pros",
    '606': "No Bail",
    '604': "Bond to Stand",
    '605': "I-Bonds",
    '277': "EMI",
    '601': "D-Bonds",
    '153': "C-Bonds",
    '894': "Electronic Monitor",
    '278': "Admit to Sheriff EM Prg",
    '279': "Not Admitted to EM Prg",
}

# EDIT THIS: Uncomment the line below and update it to so it converts the columns with codes to 
# human-readable names.

# data_january = data_january.

# CAN'T TOUCH THIS! The code below tests your solution.
def test_header_codes_renamed(df, lookup):
    for old_col_name, new_col_name in lookup.items():
        # Oh hey! You can test if a column exists in a dataframe similar to how you'd
        # check if a key is in a dictionary!
        assert new_col_name in df
        
try:
    test_header_codes_renamed(data_january, CODE_LOOKUP)
except AssertionError:
    print_red("\n⚠️ Oops! It looks like you didn't rename at least one of the columns correctly.\n")
else:
    print_green("\n👍 You got it!\n")

display(data_january.head())


⚠️ Oops! It looks like you didn't rename at least one of the columns correctly.



,date,103,606,604,605,277,601,153,TOTAL CASES,894,278,279,RELS'D ON,RELS'D ON D
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C BOND,BOND
1,011/2017,0.0,4.0,0.0,45.0,12.0,64,0.0,125,0.0,0.0,0,1,67
2,1/2/2017,0.0,4.0,0.0,37.0,9.0,68,3.0,121,0.0,0.0,0,3,63
3,1/3/2017,0.0,3.0,0.0,21.0,10.0,39,0.0,73,0.0,0.0,0,0,23
4,1/4/2017,0.0,6.0,0.0,27.0,20.0,41,0.0,94,0.0,1.0,2,0,32


If you weren't able to figure it out, here's the solution. Make sure you run this cell to get your data in the right shape for the rest of the excercise.

In [268]:
data_january = data_january.rename(columns=CODE_LOOKUP)
display(data_january.head())

,date,Nolle Pros,No Bail,Bond to Stand,I-Bonds,EMI,D-Bonds,C-Bonds,TOTAL CASES,Electronic Monitor,Admit to Sheriff EM Prg,Not Admitted to EM Prg,RELS'D ON,RELS'D ON D
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C BOND,BOND
1,011/2017,0.0,4.0,0.0,45.0,12.0,64,0.0,125,0.0,0.0,0,1,67
2,1/2/2017,0.0,4.0,0.0,37.0,9.0,68,3.0,121,0.0,0.0,0,3,63
3,1/3/2017,0.0,3.0,0.0,21.0,10.0,39,0.0,73,0.0,0.0,0,0,23
4,1/4/2017,0.0,6.0,0.0,27.0,20.0,41,0.0,94,0.0,1.0,2,0,32


### Renaming columns: Merging column names that span multiple rows

The first row of the data still contains the additional part of the column name for the latter columns. Let's merge the first two rows together.  The pseudocode for this operation goes something like this:

* Loop through the column names
* For each column name, check if there's a value in the first row
* If there isn't a value, do nothing
* If there is a value, add it to the existing column name

We'll use [`pandas.isnull`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.isnull.html) to check whether there's a value in each column of the first row.

For this particular case, it seems easier to update the columns by assigning a new value to `DataFrame.columns` instead of using `DataFrame.rename`.

In [269]:
def merge_headers(df, col_name):
    first_row_val = df[col_name][0]
    if pd.isnull(first_row_val):
        return col_name
    
    return '{0} {1}'.format(col_name, first_row_val)

# Since we need to access the dataframe in our column name mapping function,
# can't really use `pandas.DataFrame.rename` because it just passes the column
# name to the mapper. We'll just modify the list of column names and then
# assign them to the `columns` attribute of our dataframe.

# Get the current column names
col_names = data_january.columns.values.tolist()

# Map the old names to new ones that merge two rows together.
# This is called a list comprehension
new_col_names = [merge_headers(data_january, c) for c in col_names]
data_january.columns = new_col_names

display(data_january.head())

,date,Nolle Pros,No Bail,Bond to Stand,I-Bonds,EMI,D-Bonds,C-Bonds,TOTAL CASES,Electronic Monitor,Admit to Sheriff EM Prg,Not Admitted to EM Prg,RELS'D ON C BOND,RELS'D ON D BOND
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C BOND,BOND
1,011/2017,0.0,4.0,0.0,45.0,12.0,64,0.0,125,0.0,0.0,0,1,67
2,1/2/2017,0.0,4.0,0.0,37.0,9.0,68,3.0,121,0.0,0.0,0,3,63
3,1/3/2017,0.0,3.0,0.0,21.0,10.0,39,0.0,73,0.0,0.0,0,0,23
4,1/4/2017,0.0,6.0,0.0,27.0,20.0,41,0.0,94,0.0,1.0,2,0,32


### Remove rows

We've merged the header rows together, but we still have the wonky first row of our dataset. Let's get rid of the first row with [`DataFrame.drop`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html).

In [270]:
data_january = data_january.drop(0)
display(data_january.head())

,date,Nolle Pros,No Bail,Bond to Stand,I-Bonds,EMI,D-Bonds,C-Bonds,TOTAL CASES,Electronic Monitor,Admit to Sheriff EM Prg,Not Admitted to EM Prg,RELS'D ON C BOND,RELS'D ON D BOND
1,011/2017,0.0,4.0,0.0,45.0,12.0,64,0.0,125,0.0,0.0,0,1,67
2,1/2/2017,0.0,4.0,0.0,37.0,9.0,68,3.0,121,0.0,0.0,0,3,63
3,1/3/2017,0.0,3.0,0.0,21.0,10.0,39,0.0,73,0.0,0.0,0,0,23
4,1/4/2017,0.0,6.0,0.0,27.0,20.0,41,0.0,94,0.0,1.0,2,0,32
5,1/5/2017,0.0,8.0,0.0,24.0,7.0,53,0.0,92,0.0,0.0,0,0,46


### Renaming columns with a function

The column headings are now human-readable, but they still look ugly and might cause problems if we ever want to export the data and import it into another program. We can write a function that takes an existing column name as a n argument and returns a new column name. We can then pass this function to `DataFrame.rename`.

In [271]:
def slugify(s, replacement_char="_"):
    """Return version of string that can be used as column name, property or dictionary key"""
    slugified = s
    slugified = re.sub(r'[\s\-]+', replacement_char, slugified)
    slugified = slugified.lower()
    return slugified
    
data_january = data_january.rename(columns=slugify)

print(data_january.columns)

Index(['date', 'nolle_pros', 'no_bail', 'bond_to_stand', 'i_bonds', 'emi',
       'd_bonds', 'c_bonds', 'total_cases', 'electronic_monitor',
       'admit_to_sheriff_em_prg', 'not_admitted_to_em_prg', 'rels'd_on_c_bond',
       'rels'd_on_d_bond'],
      dtype='object')


### Renaming columns with a function: you try it

Things are looking better, but we forgot to deal with the apostrophe in the latter column names. Update the slugify function to remove apostrophes (`'`) and make our `data_january` `DataFrame` have really clean column names.

In [272]:
def slugify(s, replacement_char="_"):
    """Return version of string that can be used as column name, property or dictionary key"""
    slugified = s
    # EDIT THIS! Modify this function so that it removes apostrophes.
    slugified = re.sub(r'[\s\-]+', replacement_char, slugified)
    slugified = slugified.lower()
    return slugified

# EDIT THIS! Add a line below to use your `slugify` function to rename the columns.



# CAN'T TOUCH THIS! The code below tests your solution.
def test_no_apostrophes(df):
    assert df.columns[-2] == 'relsd_on_c_bond'
    assert df.columns[-1] == 'relsd_on_d_bond'

    
try:
    test_no_apostrophes(data_january)
except AssertionError:
    print_red("\n⚠️ Oops! It looks like you didn't rename at least one of the columns correctly.\n")
else:
    print_green("\n👍 You got it!\n")
    
display(data_january.head())



⚠️ Oops! It looks like you didn't rename at least one of the columns correctly.



,date,nolle_pros,no_bail,bond_to_stand,i_bonds,emi,d_bonds,c_bonds,total_cases,electronic_monitor,admit_to_sheriff_em_prg,not_admitted_to_em_prg,rels'd_on_c_bond,rels'd_on_d_bond
1,011/2017,0.0,4.0,0.0,45.0,12.0,64,0.0,125,0.0,0.0,0,1,67
2,1/2/2017,0.0,4.0,0.0,37.0,9.0,68,3.0,121,0.0,0.0,0,3,63
3,1/3/2017,0.0,3.0,0.0,21.0,10.0,39,0.0,73,0.0,0.0,0,0,23
4,1/4/2017,0.0,6.0,0.0,27.0,20.0,41,0.0,94,0.0,1.0,2,0,32
5,1/5/2017,0.0,8.0,0.0,24.0,7.0,53,0.0,92,0.0,0.0,0,0,46


If you weren't able to figure it out, here's the solution. Make sure you run this cell to get your data in the right shape for the rest of the excercise.

In [273]:
def slugify(s, replacement_char="_"):
    """Return version of string that can be used as column name, property or dictionary key"""
    slugified = s
    slugified = re.sub(r'[\s\-]+', replacement_char, slugified)
    slugified = slugified.lower()
    return slugified

data_january = data_january.rename(columns=slugify)

display(data_january)

,date,nolle_pros,no_bail,bond_to_stand,i_bonds,emi,d_bonds,c_bonds,total_cases,electronic_monitor,admit_to_sheriff_em_prg,not_admitted_to_em_prg,rels'd_on_c_bond,rels'd_on_d_bond
1,011/2017,0.0,4.0,0.0,45.0,12.0,64,0.0,125,0.0,0.0,0,1,67
2,1/2/2017,0.0,4.0,0.0,37.0,9.0,68,3.0,121,0.0,0.0,0,3,63
3,1/3/2017,0.0,3.0,0.0,21.0,10.0,39,0.0,73,0.0,0.0,0,0,23
4,1/4/2017,0.0,6.0,0.0,27.0,20.0,41,0.0,94,0.0,1.0,2,0,32
5,1/5/2017,0.0,8.0,0.0,24.0,7.0,53,0.0,92,0.0,0.0,0,0,46
6,1/6/2017,0.0,10.0,0.0,20.0,9.0,43,0.0,82,0.0,0.0,0,0,33
7,1/7/2017,0.0,7.0,4.0,39.0,18.0,53,0.0,121,0.0,0.0,0,0,56
8,1/8/2017,0.0,2.0,0.0,24.0,26.0,78,1.0,131,0.0,0.0,0,1,72
9,1/9/2017,2.0,6.0,0.0,26.0,11.0,38,0.0,83,0.0,22.0,20,1,73
10,1/10/2017,2.0,7.0,0.0,24.0,15.0,35,0.0,83,0.0,3.0,6,0,18


## Detecting a bad value

We can see the datatypes of the columns in a `DataFrame` by inspecting the `DataFrame.dtypes` attribute. When loading data, Pandas tries to convert the CSV values to sensible data types. However, if the types don't match our expectations, it might indicate that there are some bad values in our data.

One important thing to note.  The data types used by Pandas have non-obvious names. This table, which I lifted from  ["Data Types and Formats"](http://chris.friedline.net/2015-12-15-rutgers/lessons/python2/03-data-types-and-format.html) helps clear things up:

| Pandas Type | Native Python Type | Description |
| --- | --- | --- |
| object | string | The most general dtype. Will be assigned to your column if column has mixed types (numbers and strings).|
| int64 | int | Numeric characters. 64 refers to the memory allocated to hold this character. |
| float64 | float | Numeric characters with decimals. If a column contains numbers and NaNs(see below), pandas will default to float64, in case your missing value has a decimal. |
| datetime64, timedelta[ns]	| N/A (but see the datetime module in Python's standard library) | Values meant to hold time data. Look into these for time series experiments. |

In [274]:
data_january.dtypes

date                        object
nolle_pros                 float64
no_bail                    float64
bond_to_stand              float64
i_bonds                    float64
emi                        float64
d_bonds                     object
c_bonds                    float64
total_cases                 object
electronic_monitor         float64
admit_to_sheriff_em_prg    float64
not_admitted_to_em_prg      object
rels'd_on_c_bond            object
rels'd_on_d_bond            object
dtype: object

🤔 The date column seems like it should be `datetime64`, but it's not.  Let's try to convert it using [`pandas.to_datetime`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)

In [275]:
try:
    pd.to_datetime(data_january['date'])
except ValueError as e:
    msg = "Error converting values to dates: {0}".format(e)
    print(msg)

Error converting values to dates: Unknown string format


That didn't work. We can make `pandas.to_datetime` more forgiving by specifying the `errors` argument.

In [276]:
pd.to_datetime(data_january['date'], errors='coerce')

1    2017-11-01
2    2017-01-02
3    2017-01-03
4    2017-01-04
5    2017-01-05
6    2017-01-06
7    2017-01-07
8    2017-01-08
9    2017-01-09
10   2017-01-10
11   2017-01-11
12   2017-01-12
13   2017-01-13
14   2017-01-14
15   2017-01-15
16   2017-01-16
17   2017-01-17
18   2017-01-18
19   2017-01-19
20   2017-01-20
21   2017-01-21
22   2017-01-22
23   2017-01-23
24   2017-01-24
25   2017-01-25
26   2017-01-26
27   2017-01-27
28   2017-01-28
29   2017-01-29
30   2017-01-30
31   2017-01-31
32          NaT
Name: date, dtype: datetime64[ns]

It took the best guess at interpretting our wonky date string, but it didn't give us the correct value. We should be careful using `errors='coerce'`.

We can also ignore errors, which will avoid the code throwing an exception ...

In [277]:
pd.to_datetime(data_january['date'], errors='ignore')

1      011/2017
2      1/2/2017
3      1/3/2017
4      1/4/2017
5      1/5/2017
6      1/6/2017
7      1/7/2017
8      1/8/2017
9      1/9/2017
10    1/10/2017
11    1/11/2017
12    1/12/2017
13    1/13/2017
14    1/14/2017
15    1/15/2017
16    1/16/2017
17    1/17/2017
18    1/18/2017
19    1/19/2017
20    1/20/2017
21    1/21/2017
22    1/22/2017
23    1/23/2017
24    1/24/2017
25    1/25/2017
26    1/26/2017
27    1/27/2017
28    1/28/2017
29    1/29/2017
30    1/30/2017
31    1/31/2017
32       TOTALS
Name: date, dtype: object

But then none of the values get converted.

### Updating a value in-place

We need to fix our wonky value. One way we can do it is by accessing the value directly.

In [278]:
# Let's fix our original column
data_january.at[1, 'date'] = '1/1/2017'

display(data_january.head())

,date,nolle_pros,no_bail,bond_to_stand,i_bonds,emi,d_bonds,c_bonds,total_cases,electronic_monitor,admit_to_sheriff_em_prg,not_admitted_to_em_prg,rels'd_on_c_bond,rels'd_on_d_bond
1,1/1/2017,0.0,4.0,0.0,45.0,12.0,64,0.0,125,0.0,0.0,0,1,67
2,1/2/2017,0.0,4.0,0.0,37.0,9.0,68,3.0,121,0.0,0.0,0,3,63
3,1/3/2017,0.0,3.0,0.0,21.0,10.0,39,0.0,73,0.0,0.0,0,0,23
4,1/4/2017,0.0,6.0,0.0,27.0,20.0,41,0.0,94,0.0,1.0,2,0,32
5,1/5/2017,0.0,8.0,0.0,24.0,7.0,53,0.0,92,0.0,0.0,0,0,46


### Updating values with a function

But what if we had multiple errors that still followed a predictable pattern? We might write a function that can fix the values.

In [279]:
def fix_bad_date(s):
    """
    Correct a malform date string date string 
    
    Fix a string containing a date in the  form M/DD/YYYY where the `/` between
    the month and day is missing.
    
    """
    bits = s.lstrip('0').split('/')
    if len(bits) == 2:
        bits = [bits[0][0], bits[0][1:], bits[1]]
    
    return '/'.join(bits)
    
print(fix_bad_date('011/2017'))

1/1/2017


To run the value on every value in a column, we use [`Series.apply`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.apply.html). Remember, a column of a `DataFrame` is a `Series`.

In [280]:
# Undo our previous fix
data_january.at[1, 'date'] = '011/2017'

# Now run it on the entire column
data_january['date'].apply(fix_bad_date)

1      1/1/2017
2      1/2/2017
3      1/3/2017
4      1/4/2017
5      1/5/2017
6      1/6/2017
7      1/7/2017
8      1/8/2017
9      1/9/2017
10    1/10/2017
11    1/11/2017
12    1/12/2017
13    1/13/2017
14    1/14/2017
15    1/15/2017
16    1/16/2017
17    1/17/2017
18    1/18/2017
19    1/19/2017
20    1/20/2017
21    1/21/2017
22    1/22/2017
23    1/23/2017
24    1/24/2017
25    1/25/2017
26    1/26/2017
27    1/27/2017
28    1/28/2017
29    1/29/2017
30    1/30/2017
31    1/31/2017
32       TOTALS
Name: date, dtype: object

`Series.apply` returns a copy of the series with the function applied. If we want to modify our `DataFrame` we need to assign the result of `Series.apply` back to the column.

In [281]:
# Apply our transformation and assign the result back to the column
data_january['date'] = data_january['date'].apply(fix_bad_date)

# Now that we've fixed the bad value we should be able to convert our column to a date type, right?
data_january['date'] = pd.to_datetime(data_january['date'], errors='ignore')

print(data_january.dtypes)

date                        object
nolle_pros                 float64
no_bail                    float64
bond_to_stand              float64
i_bonds                    float64
emi                        float64
d_bonds                     object
c_bonds                    float64
total_cases                 object
electronic_monitor         float64
admit_to_sheriff_em_prg    float64
not_admitted_to_em_prg      object
rels'd_on_c_bond            object
rels'd_on_d_bond            object
dtype: object


That doesn't seem right. The `date` column still has the type `object`.

What's wrong? Think about it and then scroll down for the answer.

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

There's still a non-date value in the last row, `TOTALS`.

Can you filter it out and successfully convert the `date` column to a date type? This should be fine because we should be able to calculate the monthly totals ourselves. In a real-world situation, I might save the monthly totals from the source data in a separate frame to validate the other rows.

In [282]:
# EDIT THIS! Uncomment and update the line below to remove the `TOTALS` row from the `DataFrame`.

# data_january =

# EDIT THIS! Add a line of code that converts the `date` column to a date type

# CAN'T TOUCH THIS! 
def test_totals_removed(df):
    assert df['date'].iloc[-1] != 'TOTALS', "The `TOTALS` row was not removed."
    
def test_date_col_is_not_object(df):
    assert str(data_january.dtypes[0]) != 'object', "The `date` column still has a type of `object`"
    
try:
    test_totals_removed(data_january)
except AssertionError as e:
    print_red("\n⚠️ {0}\n".format(e))
else:
    print_green("\n👍 You removed the `TOTALS` row!\n")
    
try:
    test_date_col_is_not_object(data_january)
except AssertionError as e:
    print_red("\n⚠️ {0}\n".format(e))
else:
    print_green("\n👍 The `date` column has a date type. Nice work!\n")


⚠️ The `TOTALS` row was not removed.


⚠️ The `date` column still has a type of `object`



If you weren't able to figure it out, here's the solution. Make sure you run this cell to get your data in the right shape for the rest of the excercise.

In [283]:
data_january = data_january[data_january['date'] != 'TOTALS']
data_january['date'] = pd.to_datetime(data_january['date'])

display(data_january)

,date,nolle_pros,no_bail,bond_to_stand,i_bonds,emi,d_bonds,c_bonds,total_cases,electronic_monitor,admit_to_sheriff_em_prg,not_admitted_to_em_prg,rels'd_on_c_bond,rels'd_on_d_bond
1,2017-01-01,0.0,4.0,0.0,45.0,12.0,64,0.0,125,0.0,0.0,0,1,67
2,2017-01-02,0.0,4.0,0.0,37.0,9.0,68,3.0,121,0.0,0.0,0,3,63
3,2017-01-03,0.0,3.0,0.0,21.0,10.0,39,0.0,73,0.0,0.0,0,0,23
4,2017-01-04,0.0,6.0,0.0,27.0,20.0,41,0.0,94,0.0,1.0,2,0,32
5,2017-01-05,0.0,8.0,0.0,24.0,7.0,53,0.0,92,0.0,0.0,0,0,46
6,2017-01-06,0.0,10.0,0.0,20.0,9.0,43,0.0,82,0.0,0.0,0,0,33
7,2017-01-07,0.0,7.0,4.0,39.0,18.0,53,0.0,121,0.0,0.0,0,0,56
8,2017-01-08,0.0,2.0,0.0,24.0,26.0,78,1.0,131,0.0,0.0,0,1,72
9,2017-01-09,2.0,6.0,0.0,26.0,11.0,38,0.0,83,0.0,22.0,20,1,73
10,2017-01-10,2.0,7.0,0.0,24.0,15.0,35,0.0,83,0.0,3.0,6,0,18


Pandas also provides utility methods to do other type conversions.

In [284]:
pd.to_numeric(data_january['nolle_pros'], downcast='integer')

1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     2
10    2
11    0
12    2
13    1
14    0
15    0
16    0
17    3
18    3
19    1
20    0
21    0
22    0
23    2
24    2
25    0
26    1
27    1
28    0
29    0
30    6
31    0
Name: nolle_pros, dtype: int8

## Exploratory data visualization